In [ ]:
import json


In [ ]:
#Load json file
#Returns list -> called an object 
def loadJson(_fileName):
    pass

In [ ]:
#Pass it an object
#Returns object
def combineFiles(_obj):
    pass

In [ ]:
#Remove list item
#Returns object
def removeItem(_listNumber):
    pass

In [ ]:
#Pass it an object
#Must combine inputs and output
#Format on can be any json key (fft, time series, etc)
#Saves to file
def formatForTraining(_obj, _fileName, _formatOn):
    pass

In [ ]:
#Pass it an object a list describing the data
# Example [3, 4] -> First 3 are one type (woof) and last 4 are a different type (meow)
#Retunrs plot figure
def plotObj(_obj, _listDiffTypes, _formatOn):
    pass

In [ ]:
#Define 
fileName = 'dataWoofs.json'

#Load woofs
woofs = json.load(fileName)

#Load meows
fileName = 'dataWoofs.json'
meows = json.load(fileName)

#Combine files
woofsAndMeows = combineFiles([woofs, meows])

#Plot data
plot = fig()
plotObj(woofsAndMeows, [len(woofs), len(meows)], 'f.fData')

#Export 
formatForTraining(woofsAndMeows, 'woofsAndMeows.data', 'f.fData')